In [3]:
import pandas as pd
import json

In [4]:
df = pd.read_csv("/data/raw_stock_indicator.csv")

In [5]:
def json_parser(x):
    try:
        return json.loads(x)["symbol"]
    except Exception as e:
        print(e)
        return ""

In [6]:
df["company_symbol"] = df["info_json"].map(lambda x: json_parser(x))  

In [37]:
df["company_symbol"].unique()

array(['LTIM.NS', '20MICRONS.NS', '21STCENMGM.NS', ..., 'ZODIACLOTH.NS',
       'ZOTA.NS', 'ZUARI.NS'], dtype=object)

In [15]:
df.columns

Index(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA', 'Total Unusual Items',
       'Total Unusual Items Excluding Goodwill',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBIT',
       'Net Interest Income', 'Interest Expense', 'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Diluted Average Shares', 'Basic Average Shares',
       'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Otherunder Preferred Stock Dividend',
       'Net Income', 'Minority Interests',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Non Operating Income Expenses', 'Special Income Charges',
       'Restructuring And Mergern Acquisition',
       'Net Non Operating Interest Inco

In [25]:
cols = ["Total Revenue", "Operating Revenue", "Gross Profit", "Year", "Month" ,"company_symbol", "Open", "Volume"]

In [32]:
df1 = df[cols]
df1["year_month"] = (10*df1["Year"])+df1["Month"]
df1 = df1.sort_values(by=['company_symbol', 'year_month'])
df1 

<ipython-input-32-02234de0590b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["year_month"] = (10*df1["Year"])+df1["Month"]


,Total Revenue,Operating Revenue,Gross Profit,Year,Month,company_symbol,Open,Volume,year_month
8,1.859114e+09,1.859114e+09,8.814420e+08,2022,6,20MICRONS.NS,68.783986,42835.0,20226
7,1.877775e+09,1.877775e+09,8.909090e+08,2022,9,20MICRONS.NS,94.857335,164045.0,20229
6,1.547142e+09,1.547142e+09,7.191890e+08,2022,12,20MICRONS.NS,NaN,NaN,20232
5,1.732841e+09,1.732841e+09,7.499020e+08,2023,3,20MICRONS.NS,70.522207,617891.0,20233
13,-3.918900e+07,-3.918900e+07,-4.288500e+07,2022,6,21STCENMGM.NS,25.049999,3903.0,20226
...,...,...,...,...,...,...,...,...,...
7314,1.295920e+10,1.295920e+10,3.533500e+09,2022,6,ZUARI.NS,144.750000,82964.0,20226
7313,5.016200e+09,5.016200e+09,1.144400e+09,2022,9,ZUARI.NS,168.600006,457551.0,20229
7312,1.416170e+10,1.416170e+10,5.367000e+09,2022,12,ZUARI.NS,NaN,NaN,20232
7311,1.338910e+10,1.338910e+10,4.684400e+09,2023,3,ZUARI.NS,121.900002,98839.0,20233


In [53]:
#Old to new Price
df1['price_old'] = df1.groupby(['company_symbol'])['Open'].shift(1)
df1['price_change'] = df1['Open'] / df1['price_old']

#Old to new revenue
df1['revenue_old'] = df1.groupby(['company_symbol'])['Total Revenue'].shift(1)
df1['revenue_change'] = df1['Total Revenue'] / df1['revenue_old']

#Next quarter price
df1["price_target"] = df1.groupby(['company_symbol'])['Open'].shift(-1)
df1['price_target_change'] = df1['price_target'] / df1['Open']

In [55]:
df1[df1["company_symbol"] == df1.iloc[2104].company_symbol]

,Total Revenue,Operating Revenue,Gross Profit,Year,Month,company_symbol,Open,Volume,year_month,price_change,price_old,revenue_old,revenue_change,price_new,price_target,price_target_change
2075,76314000.0,76314000.0,21127000.0,2022,6,GANGAFORGE.NS,5.75,90316.0,20226,NaN,NaN,NaN,NaN,5.70,5.70,0.991304
2074,92085000.0,92085000.0,27817000.0,2022,9,GANGAFORGE.NS,5.70,156206.0,20229,0.991304,5.75,76314000.0,1.206659,NaN,NaN,NaN
2073,81324000.0,81324000.0,27669000.0,2022,12,GANGAFORGE.NS,NaN,NaN,20232,NaN,5.70,92085000.0,0.883141,3.05,3.05,NaN
2072,84373000.0,84373000.0,33785000.0,2023,3,GANGAFORGE.NS,3.05,178263.0,20233,NaN,NaN,81324000.0,1.037492,5.90,5.90,1.934426
2071,98596000.0,98596000.0,26774000.0,2023,6,GANGAFORGE.NS,5.90,548103.0,20236,1.934426,3.05,84373000.0,1.168573,NaN,NaN,NaN


In [56]:
df1["Good_stock"] = df1["price_target_change"] > 1.5

In [59]:
df1[df1["Good_stock"]]

,Total Revenue,Operating Revenue,Gross Profit,Year,Month,company_symbol,Open,Volume,year_month,price_change,price_old,revenue_old,revenue_change,price_new,price_target,price_target_change,Good_stock
65,1.066354e+09,1.066354e+09,3.425530e+08,2022,6,AARVI.NS,88.619374,2649.0,20226,NaN,NaN,NaN,NaN,143.750000,143.750000,1.622106,True
108,4.084425e+11,4.084425e+11,7.297020e+10,2022,6,ADANIENT.NS,2216.839183,1568415.0,20226,NaN,NaN,NaN,NaN,3444.504377,3444.504377,1.553791,True
215,5.378278e+08,5.378278e+08,1.894442e+08,2022,6,AIROLAM.NS,63.000000,3446.0,20226,NaN,NaN,NaN,NaN,113.800003,113.800003,1.806349,True
234,2.926560e+08,2.926560e+08,9.375900e+07,2022,6,AKASH.NS,23.692215,129876.0,20226,NaN,NaN,NaN,NaN,37.150002,37.150002,1.568026,True
238,4.631110e+08,4.631110e+08,2.856200e+07,2022,6,AKG.NS,36.150002,37808.0,20226,NaN,NaN,NaN,NaN,63.549999,63.549999,1.757953,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7244,6.780000e+09,6.780000e+09,3.720000e+09,2023,3,WOCKPHARMA.NS,151.350006,1372837.0,20233,NaN,NaN,6.990000e+09,0.969957,236.100006,236.100006,1.559960,True
7252,1.494235e+09,1.494235e+09,1.353398e+09,2022,6,WONDERLA.NS,230.768724,350157.0,20226,NaN,NaN,NaN,NaN,373.293534,373.293534,1.617609,True
7164,5.070320e+08,5.070320e+08,-2.530400e+07,2023,3,WSI.NS,32.549999,3020.0,20233,NaN,NaN,2.494330e+08,2.032738,101.050003,101.050003,3.104455,True
7217,1.114150e+10,1.114150e+10,6.103969e+09,2022,6,WSTCSTPAPR.NS,321.240533,283105.0,20226,NaN,NaN,NaN,NaN,491.727850,491.727850,1.530715,True


In [61]:
df1.to_csv("/data/prepared_stock_indicator.csv", index=False)